In [1]:
from experiments.config import get_default_config
from src.benchmark.synthetic_dataset import  get_task,get_dataloader
from src.libs.soi import  SOI
from src.libs.soi_grad import  SOI_grad
import numpy as np

In [2]:
## Default config
args=get_default_config()

### Redundancy

In [3]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"red","nb":3} ]
args.dim = 1
task_red = get_task(args,my_settings)
task_red.o_inf()

0.14736860225060955

### Synergy

In [4]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"syn","nb":3} ]
args.dim = 1
task_syn = get_task(args,my_settings)
task_syn.get_summary()["o_inf"]

-0.12391808195228782

### Redundancy + Synergy

In [5]:
## Make a synthetic task with 6 made of two redundant subtasks of 3 variables.
my_settings = [{"rho":0.6,"type":"red","nb":3},{"rho":0.6,"type":"syn","nb":3} ]
args.dim = 1
task_both = get_task(args,my_settings)
task_both.get_summary()["o_inf"]

0.02345052029832928

# O-information estimation

### General config

In [6]:
args.use_ema=True
args.weight_s_functions = True
args.importance_sampling = True
args.bs = 512
args.warmup_epochs = 30
args.max_epochs = 50
args.test_epoch = 5
args.debug = True
args.ou_dir = "quickstart/"

### Runing SOI 

### On the redundancy benchmark

In [7]:
gt = task_red.get_summary()
train_l, test_l  = get_dataloader(task_red,args)
## Run SOI
soi = SOI(args, nb_var = task_red.nb_var, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
results = soi.compute_o_inf()
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )


Weighting the scores to learn 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 623 K 
1 | model_ema | EMA  | 623 K 
-----------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.984     Total estimated model params size (MB)


Sanity Checking: |                                                    | 0/? [00:00<?, ?it/s]

Training: |                                                           | 0/? [00:00<?, ?it/s]

Epoch:  35  GT:  0.147 SOI_estimate:  0.143
Epoch:  40  GT:  0.147 SOI_estimate:  0.146
Epoch:  45  GT:  0.147 SOI_estimate:  0.143


Validation: |                                                         | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


SOI: {' O-inf': 0.1467235326766968, 'tc': 0.4903195172548294, 'dtc': 0.3435959845781326, 'S-info': 0.833915501832962}
GT: {' O-inf': 0.14736860225060955, 'tc': 0.5220620516920196, 'dtc': 0.37469344944141003, 'S-info': 0.8967555011334296}


### On the synergy benchmark

In [9]:
gt = task_syn.get_summary()
train_l, test_l  = get_dataloader(task_syn,args)
## Run SOI
soi = SOI(args, nb_var = task_syn.nb_var, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
results = soi.compute_o_inf()
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )


Weighting the scores to learn 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 623 K 
1 | model_ema | EMA  | 623 K 
-----------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.984     Total estimated model params size (MB)


Sanity Checking: |                                                    | 0/? [00:00<?, ?it/s]

Training: |                                                           | 0/? [00:00<?, ?it/s]

Epoch:  35  GT:  -0.124 SOI_estimate:  -0.125
Epoch:  40  GT:  -0.124 SOI_estimate:  -0.126
Epoch:  45  GT:  -0.124 SOI_estimate:  -0.11


Validation: |                                                         | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


SOI: {' O-inf': -0.14697401523590087, 'tc': 0.49834243655204774, 'dtc': 0.6453164517879486, 'S-info': 1.1436588883399963}
GT: {' O-inf': -0.12391808195228782, 'tc': 0.5697171415941797, 'dtc': 0.6936352235464676, 'S-info': 1.2633523651406473}


### On the mixed benchmark

In [15]:
gt = task_both.get_summary()
train_l, test_l  = get_dataloader(task_both,args)
## Run SOI
args.warmup_epochs = 60
args.max_epochs = 80

soi = SOI(args, nb_var = task_both.nb_var, gt = gt)
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
results = soi.compute_o_inf()
print("SOI:", {" O-inf": results["o_inf"],"tc":results["tc"], "dtc":results["dtc"], "S-info": results["s_inf"] })
print("GT:", {" O-inf": gt["o_inf"],"tc":gt["tc"], "dtc":gt["dtc"], "S-info": gt["s_inf"] } )

Weighting the scores to learn 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 624 K 
1 | model_ema | EMA  | 624 K 
-----------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.993     Total estimated model params size (MB)


Sanity Checking: |                                                    | 0/? [00:00<?, ?it/s]

Training: |                                                           | 0/? [00:00<?, ?it/s]

Validation: |                                                         | 0/? [00:00<?, ?it/s]

Epoch:  65  GT:  0.023 SOI_estimate:  0.056
Epoch:  70  GT:  0.023 SOI_estimate:  0.03
Epoch:  75  GT:  0.023 SOI_estimate:  0.071


`Trainer.fit` stopped: `max_epochs=80` reached.


SOI: {' O-inf': 0.04701898694038398, 'tc': 1.0358725666999817, 'dtc': 0.9888535797595978, 'S-info': 2.0247261464595794}
GT: {' O-inf': 0.02345052029832928, 'tc': 1.0917791932861967, 'dtc': 1.0683286729878674, 'S-info': 2.160107866274064}


## Gradients of O-information estimation

In [7]:
gt = task_both.get_summary()
print("g_o_ing : "," ".join([ "x{}: {},".format(i,np.round(x,decimals=3) ) for i,x in enumerate(gt["g_o_inf"]) ]) )

g_o_ing :  x0: 0.147, x1: 0.147, x2: 0.147, x3: -0.124, x4: -0.124, x5: -0.124,


In [8]:
## Run SOI with gradient of o-information estimation
args.warmup_epochs = 80
args.max_epochs = 100
## Run SOI
gt = task_both.get_summary()
train_l, test_l  = get_dataloader(task_both,args)
soi = SOI_grad(args, nb_var = task_both.nb_var, gt = task_both.get_summary())
## Fit the model
soi.fit(train_l, test_l)
## Compute O_information using the test_loader
print("The final estimation :", soi.compute_o_inf_with_grad() )

Weighting the scores to learn 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type | Params
-----------------------------------
0 | score     | DiT  | 624 K 
1 | model_ema | EMA  | 624 K 
-----------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.993     Total estimated model params size (MB)


Sanity Checking: |                                                    | 0/? [00:00<?, ?it/s]

Training: |                                                           | 0/? [00:00<?, ?it/s]

Validation: |                                                         | 0/? [00:00<?, ?it/s]

O_inf - GT:  0.023 O_inf - SOI_new:  0.044
Gradient O_inf - GT:  x0: 0.147, x1: 0.147, x2: 0.147, x3: -0.124, x4: -0.124, x5: -0.124,
Gradient O_inf - SOI: x0: 0.146, x1: 0.153, x2: 0.133, x3: -0.125, x4: -0.095, x5: -0.168,
O_inf - GT:  0.023 O_inf - SOI_new:  0.041
Gradient O_inf - GT:  x0: 0.147, x1: 0.147, x2: 0.147, x3: -0.124, x4: -0.124, x5: -0.124,
Gradient O_inf - SOI: x0: 0.132, x1: 0.14, x2: 0.135, x3: -0.119, x4: -0.089, x5: -0.156,
O_inf - GT:  0.023 O_inf - SOI_new:  0.018
Gradient O_inf - GT:  x0: 0.147, x1: 0.147, x2: 0.147, x3: -0.124, x4: -0.124, x5: -0.124,
Gradient O_inf - SOI: x0: 0.143, x1: 0.124, x2: 0.126, x3: -0.138, x4: -0.136, x5: -0.174,


Validation: |                                                         | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


{'tc': 1.01420716047287,
 'dtc': 1.0057169377803803,
 'o_inf': 0.008490222692489668,
 's_inf': 2.0199240982532505,
 'g_o_inf': {'x0': 0.13632701635360722,
  'x1': 0.11200442314147951,
  'x2': 0.11321437358856212,
  'x3': -0.11913098394870753,
  'x4': -0.1387510180473327,
  'x5': -0.17350099682807918},
 'tc_minus': {'x0': 0.6894628465175628,
  'x1': 0.7175222396850586,
  'x2': 0.7158963143825531,
  'x3': 0.592874801158905,
  'x4': 0.4880099058151245,
  'x5': 0.8521396815776825},
 'dtc_minus': {'x0': 0.8172996401786804,
  'x1': 0.8210364401340484,
  'x2': 0.8206204652786255,
  'x3': 0.4652535945177078,
  'x4': 0.3407686650753021,
  'x5': 0.6701484620571136}}